In [23]:
#  Import libraries and define variables
import os
import pandas as pd
import shutil
import glob
from tqdm import tqdm
from PIL import Image
# import nibabel as nib
import numpy as np
import pickle
# import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D, Dense, Flatten, Dropout, LeakyReLU, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
from keras.preprocessing.image import ImageDataGenerator

# Define the modalities and classifications
# modalities = ['t1', 't1gd', 't2', 'flair']
# classifications = ['MGMT_positive', 'MGMT_negative']
classifications = ['benign', 'malignant']

# Define patch size and stride
block_h, block_w = (64, 64)
stride = 2

# Interpolated image dimestions
inter_dim = (110, 90)

# Define epoch
epoch = 10

# Define paths to the BraTS dataset folders
path = 'F:/gliobastoma-mgmt-upenn/'

# PATH = path + 'Data/Working_Data'
# Org_Dir = PATH + 'Original_Data_Backup/'
Work_Dir = path + 'Data/Skin Cancer/Working_Data/'

In [10]:
# Function Defination --> Load the data

def load_data():
    print('Loading data...')
    pickle_in = open(Work_Dir + 'X.pickle', 'rb')
    X = pickle.load(pickle_in) # Loading Features

    pickle_in = open(Work_Dir + 'y.pickle', 'rb')
    y = pickle.load(pickle_in) # Loading Labels 

    return X, y

In [29]:
#  Funtion Defination --> Model defination
def Model(img_h, img_w, X_train, y_train, X_test, y_test, epoch):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',kernel_initializer='he_normal',
              input_shape=(img_h, img_w, 1)))
    model.add(LeakyReLU(alpha=0.1))
    # Pool_size is the size of filter.
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Conv2D(48, (3, 3), padding='same',kernel_initializer='he_normal',))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Conv2D(48, (3, 3), padding='same',kernel_initializer='he_normal',))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Flatten())  # Convert 3D feature map to 1D feature vector.

    model.add(Dense(1096))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.1))
    model.add(Dense(2, activation='softmax'))

    # Compiling the model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    model.optimizer.learning_rate = 0.0001

    # Training the model & storing its detail in the variable h
    history = model.fit(X_train, to_categorical(y_train,num_classes=2), batch_size=128, epochs=epoch, verbose=2,
                        validation_data=(X_test, to_categorical(y_test)), shuffle=True)

    # Evaluating performance of the Model
    test_loss, test_acc = model.evaluate(X_test, to_categorical(y_test))
    results.append(test_acc)
    print('Test Loss is: ', test_loss, '\nTest Accuracy is: ', test_acc)

    # Saving the model in hierarchical Data Formate (HDF)
    model.save(path + 'Outputs/latest_model'+'.h5')

    return history, model



In [12]:
# Function Definition --> For Saving Validation Accuracy in the file

def Saving_Test_Accuracy():
    import numpy as np

    print('Saving Test Accuracy...')

    file = open(Result_File, "a")
    file.write(case)
    file.write(':\n======')
    file.write('\nAll Test Accuracy:\t')

    for ans in results:
        file.write(str(ans) + ', ')

    file.write('\nMean Test Accuracy:\t' + str(np.mean(results)))
    file.write('\nMin Test Accuracy:\t' + str(np.min(results)))
    file.write('\nMax Test Accuracy:\t' + str(np.max(results)))
    file.write('\n\n\n')
    file.close()


In [19]:
# Main cell to execute the functions

X, y = load_data()
y=np.array(y)
X=np.array(X)
# print(X[0].shape)
results = []

# Split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # Training the model
X

Loading data...


array([[[134, 138, 143, ..., 162, 161, 161],
        [131, 136, 141, ..., 162, 164, 167],
        [129, 134, 139, ..., 160, 165, 171],
        ...,
        [ 60,  69,  83, ..., 158, 157, 160],
        [ 53,  62,  75, ..., 160, 161, 159],
        [ 48,  57,  69, ..., 163, 165, 161]],

       [[152, 153, 154, ..., 166, 162, 160],
        [153, 155, 157, ..., 165, 161, 159],
        [153, 156, 160, ..., 162, 159, 156],
        ...,
        [148, 149, 151, ..., 150, 145, 141],
        [144, 146, 149, ..., 152, 147, 144],
        [142, 145, 148, ..., 155, 151, 148]],

       [[155, 155, 155, ..., 157, 154, 154],
        [159, 158, 157, ..., 155, 153, 153],
        [161, 161, 159, ..., 153, 153, 155],
        ...,
        [153, 155, 157, ..., 142, 147, 152],
        [151, 154, 156, ..., 143, 145, 149],
        [148, 151, 154, ..., 147, 147, 148]],

       ...,

       [[159, 162, 166, ..., 176, 176, 176],
        [159, 162, 166, ..., 177, 178, 178],
        [159, 162, 165, ..., 178, 179, 180

In [20]:
X=np.array(X)
X

array([[[134, 138, 143, ..., 162, 161, 161],
        [131, 136, 141, ..., 162, 164, 167],
        [129, 134, 139, ..., 160, 165, 171],
        ...,
        [ 60,  69,  83, ..., 158, 157, 160],
        [ 53,  62,  75, ..., 160, 161, 159],
        [ 48,  57,  69, ..., 163, 165, 161]],

       [[152, 153, 154, ..., 166, 162, 160],
        [153, 155, 157, ..., 165, 161, 159],
        [153, 156, 160, ..., 162, 159, 156],
        ...,
        [148, 149, 151, ..., 150, 145, 141],
        [144, 146, 149, ..., 152, 147, 144],
        [142, 145, 148, ..., 155, 151, 148]],

       [[155, 155, 155, ..., 157, 154, 154],
        [159, 158, 157, ..., 155, 153, 153],
        [161, 161, 159, ..., 153, 153, 155],
        ...,
        [153, 155, 157, ..., 142, 147, 152],
        [151, 154, 156, ..., 143, 145, 149],
        [148, 151, 154, ..., 147, 147, 148]],

       ...,

       [[159, 162, 166, ..., 176, 176, 176],
        [159, 162, 166, ..., 177, 178, 178],
        [159, 162, 165, ..., 178, 179, 180

In [21]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(13171, 64, 64)
(13171,)
(5645, 64, 64)
(5645,)


In [30]:
history, model = Model(block_h, block_w, X_train, y_train, X_test, y_test, epoch)


Epoch 1/10
103/103 - 43s - loss: 25.4335 - accuracy: 0.5264 - val_loss: 2.8575 - val_accuracy: 0.6243 - 43s/epoch - 419ms/step
Epoch 2/10
103/103 - 45s - loss: 7.4500 - accuracy: 0.5460 - val_loss: 2.4695 - val_accuracy: 0.5095 - 45s/epoch - 441ms/step
Epoch 3/10
103/103 - 47s - loss: 3.3289 - accuracy: 0.5413 - val_loss: 0.6916 - val_accuracy: 0.6512 - 47s/epoch - 460ms/step
Epoch 4/10
103/103 - 47s - loss: 1.8930 - accuracy: 0.5579 - val_loss: 0.8085 - val_accuracy: 0.6728 - 47s/epoch - 458ms/step
Epoch 5/10
103/103 - 46s - loss: 1.4215 - accuracy: 0.5718 - val_loss: 0.6905 - val_accuracy: 0.6034 - 46s/epoch - 451ms/step
Epoch 6/10
103/103 - 47s - loss: 1.2538 - accuracy: 0.5614 - val_loss: 0.6675 - val_accuracy: 0.6117 - 47s/epoch - 456ms/step
Epoch 7/10
103/103 - 49s - loss: 1.1226 - accuracy: 0.5635 - val_loss: 0.7065 - val_accuracy: 0.5979 - 49s/epoch - 477ms/step
Epoch 8/10
103/103 - 46s - loss: 1.0390 - accuracy: 0.5637 - val_loss: 0.6247 - val_accuracy: 0.6717 - 46s/epoch - 44